<h1>Preprocessing </h1>

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
import sys # for debugging 

In [2]:

# This function preprocesses the image by reading in the image apply grayscale make all the sizes the same and 
def preprocess_image(file_path, img_size):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # Grayscale will even the playing field if we start getting different types of images. If the images color is a factor we can take out grayscale
    img = cv2.resize(img, img_size)
    img = img.astype('float')/255.0 # Make the pixels become float and normalize to 0-1 for normalization
    return img


target_size =(224, 224)

# This function will pull from the directory and all subdirectory for the image and give it a label to the directory it is in
def load_images_from_directory(directory):
    images = []
    labels = []
    # Iterates through all subdirectories
    for subdir in os.listdir(directory):
        label = subdir #Make the subdirectory name be a label
        subdir_path = os.path.join(directory, subdir)

        # Checks if the object it is looking at is a directory and if it is go into the directory and get all the files and preprocess them
        if os.path.isdir(subdir_path):
            for image in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, image)

                image = preprocess_image(file_path, target_size)

                # Append to the arrays after preprocessing
                images.append(image)
                labels.append(label)

    return np.array(images), np.array(labels)




In [3]:


# Define the directory paths for the training and test datasets
train_dir = "./Alzheimer_s Dataset/train"
test_dir = "./Alzheimer_s Dataset/test"

# Load images and labels from the training directory
alz_images_train, alz_labels_train = load_images_from_directory(train_dir)

# Load images and labels from the test directory
alz_images_test, alz_labels_test = load_images_from_directory(test_dir)

# Print information about the training dataset
print("Train")
print('Image shape:', alz_images_train.shape)
print('Labels shape:', alz_labels_train.shape)

# Print information about the test dataset
print("\nTest")
print('Image shape:', alz_images_test.shape)
print('Labels shape:', alz_labels_test.shape)


# np.set_printoptions(threshold=sys.maxsize) # for debugging

# print('Image train:', alz_images_train) # for debugging

# The output of the shape follows this
#  (X, X1, X2)
# X is the number of pictures in the array   
# X1 is the number of rows for a single picture (should be 224 since that is the scale)
# X2 is the number of columns in each picture  (should be 224 since that is the scale)
#  *Scale can be change to 207 since that is how the data is processed. 
# 
# When pull out the full array, you see alot of 0 at the start and end and that is because of the black around the brain
# 


Train
Image shape: (5121, 224, 224)
Labels shape: (5121,)

Test
Image shape: (1279, 224, 224)
Labels shape: (1279,)


<h1> Aaron's Algorithm </h1>
CNN GCNN or similar neural networks that can be adjusted in between each other

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

alz_labels_train_encoded = label_encoder.fit_transform(alz_labels_train)
alz_labels_test_encoded = label_encoder.fit_transform(alz_labels_test)

num_classes = len(label_encoder.classes_)

alz_labels_train_onehot = tf.keras.utils.to_categorical(alz_labels_train_encoded, num_classes)
alz_labels_test_onehot = tf.keras.utils.to_categorical(alz_labels_test_encoded, num_classes)

#np.set_printoptions(threshold=sys.maxsize) # for debugging
#print(alz_labels_train_onehot)

print("Training labels shape (one-hot encoded):", alz_labels_train_onehot.shape)
print("Testing labels shape (one-hot encoded):", alz_labels_test_onehot.shape)

# print('Image train:', alz_images_train) # for debugging


# 0 = MildDemented
# 1 = ModerateDemented
# 2 = NonDemented
# 3 = VeryMildDemented

Training labels shape (one-hot encoded): (5121, 4)
Testing labels shape (one-hot encoded): (1279, 4)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

img_height = target_size[1]
img_width = target_size[0]
num_channels = 1


model3 = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, num_channels)), #all factors can be adjusted
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.7),
    Dense(num_classes, activation='softmax')
])

model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 186624)            0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [10]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model3.fit(alz_images_train, alz_labels_train_onehot, epochs=10, batch_size=32, validation_data=(alz_images_test, alz_labels_test_onehot))




Epoch 1/10
161/161 [==============================] - 55s 336ms/step - loss: 1.1589 - accuracy: 0.4733 - val_loss: 1.0713 - val_accuracy: 0.3534
Epoch 2/10
161/161 [==============================] - 55s 340ms/step - loss: 1.0159 - accuracy: 0.5026 - val_loss: 1.0826 - val_accuracy: 0.5020
Epoch 3/10
161/161 [==============================] - 55s 341ms/step - loss: 0.9593 - accuracy: 0.5247 - val_loss: 1.2423 - val_accuracy: 0.5020
Epoch 4/10
161/161 [==============================] - 54s 339ms/step - loss: 0.8850 - accuracy: 0.5688 - val_loss: 1.0281 - val_accuracy: 0.5465
Epoch 5/10
161/161 [==============================] - 52s 323ms/step - loss: 0.8115 - accuracy: 0.6009 - val_loss: 0.9684 - val_accuracy: 0.5504
Epoch 6/10
161/161 [==============================] - 54s 333ms/step - loss: 0.7391 - accuracy: 0.6317 - val_loss: 1.1631 - val_accuracy: 0.5590
Epoch 7/10
161/161 [==============================] - 53s 329ms/step - loss: 0.6418 - accuracy: 0.6917 - val_loss: 1.1526 - val_ac

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

img_height = target_size[1]
img_width = target_size[0]
num_channels = 1


model4 = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, num_channels)), #all factors can be adjusted
    MaxPooling2D((2,2)),
    # Conv2D(64, (3,3), activation='relu'),
    # MaxPooling2D((2,2)),

    Flatten(),

    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model4.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 394272)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               50466944  
                                                                 
 dense_3 (Dense)             (None, 4)                 516       
                                                                 
Total params: 50,467,780
Trainable params: 50,467,780
Non-trainable params: 0
__________________________________________

In [8]:
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model4.fit(alz_images_train, alz_labels_train_onehot, epochs=10, batch_size=32, validation_data=(alz_images_test, alz_labels_test_onehot))

Epoch 1/10
161/161 [==============================] - 40s 246ms/step - loss: 1.9952 - accuracy: 0.5427 - val_loss: 0.9959 - val_accuracy: 0.5262
Epoch 2/10
161/161 [==============================] - 39s 245ms/step - loss: 0.5251 - accuracy: 0.8125 - val_loss: 1.0899 - val_accuracy: 0.6091
Epoch 3/10
161/161 [==============================] - 40s 246ms/step - loss: 0.1928 - accuracy: 0.9596 - val_loss: 1.2683 - val_accuracy: 0.6099
Epoch 4/10
161/161 [==============================] - 40s 247ms/step - loss: 0.0609 - accuracy: 0.9955 - val_loss: 1.2875 - val_accuracy: 0.6106
Epoch 5/10
161/161 [==============================] - 39s 245ms/step - loss: 0.0282 - accuracy: 0.9986 - val_loss: 1.4521 - val_accuracy: 0.6411
Epoch 6/10
161/161 [==============================] - 40s 246ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 1.6972 - val_accuracy: 0.6177
Epoch 7/10
161/161 [==============================] - 40s 249ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 1.9019 - val_ac

<h1>Jay's Algorthm</h1>
SVM and KNN (K-Nearest Neighbors)

<h1>Geoffrey's Algorithm</h1>
Random Forest and RNN

Notes below

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Print model summary
model.summary()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Initialize the CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, num_channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Add flattening layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Print model summary
model.summary()


This for DENSE (fully connected layer)

Tunable Hyperparameters: Instead of choosing a fixed number of units, you can treat the number of units in dense layers as a hyperparameter to be tuned during model training using techniques like grid search or random search. This approach allows for more flexibility in optimizing the model's architecture for specific datasets and tasks.

In [ ]:
#visualization and accuracy code below

# test_loss, test_accuracy = model1.evaluate(alz_images_test, alz_labels_test_onehot)
# print('Test Loss:', test_loss)
# print('Test Accuracy:', test_accuracy)

# import matplotlib.pyplot as plt

# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend(loc='lower right')
# plt.show()